# Modules

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import os
import csv
import random
import requests
import time

import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time
import random
from fake_useragent import UserAgent

from datetime import datetime, date
import re
from collections import defaultdict

import numpy as np
import openpyxl

In [2]:
def make_full_excel_path(tiker):
    basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports'
    full_path = basic_folder +'\\' + tiker
    return full_path

## Selenium Options

In [152]:
def selelenium(tiker=''):
    basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports'
    full_path = make_full_excel_path(tiker)

    user_agent = UserAgent()

    option = webdriver.FirefoxOptions()
    option.set_preference('dom.webdriver.enabled', False)
    option.add_argument(f'user-agent={user_agent.random}')
    option.set_preference('dom.webnotifications.enebled', False)
    option.set_preference('media.volume_scale', '0.0')

    option.set_preference("browser.download.folderList", 2)
    option.set_preference("browser.download.manager.showWhenStarting", False)

    option.set_preference("browser.download.dir", full_path)
#     option.set_preference("browser.helperApps.neverAsk.saveToDisk","text/xlsx")
    option.set_preference('browser.helperApps.neverAsk.saveToDisk',
                          'text/plain, application/vnd.ms-excel, text/csv, text/comma-separated-values, application/octet-stream, application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    option.headless = False  # True - скрывает отображение работы браузера

    return webdriver.Firefox(executable_path=r"C:\Users\nickl\3. Work Projects\Market_go\geckodriver.exe", options=option)

In [153]:
def RST():
    ''' Seg.gov often blocks you. And maybe this little things can halpe. '''
    return time.sleep(random.randint(3, 6))

# Search Reports or Download from Sec.Gov
### Output = Launch download excels Reports
1. find_cik(tiker)
2. find_all_reports(CIK,periods=20)
3. def convert_from_xmls(xml_report_urls, tiker) 
4. def main_func(tiker)
basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports'
5.def open_folder_with_excels(tiker):

In [5]:
def find_cik(tiker):  # '1013934'
    browser = selelenium()

    browser.get('https://sec.report/CIK/')
    time.sleep(3)
    browser.find_element_by_xpath(
        '/html/body/div/div/table[2]/tbody/tr[2]/td[1]/input').send_keys(tiker)
    time.sleep(3)
    browser.find_element_by_xpath(
        '/html/body/div/div/table[2]/tbody/tr[2]/td[2]/input').click()
    time.sleep(3)
    cik = browser.find_elements_by_tag_name("h2")[0].text
    browser.close()
    browser.quit()
    return cik[len(cik) - 7:]

In [6]:
def find_all_reports(CIK,periods=16): # сслыки на отчетности ['https://www.sec.gov/ix?doc=/Archives/edgar/data/0001013934/000101393421000008/stra-20210331.htm',...]
    xml_report_urls = []
    
    browser = selelenium()
    browser.get(f'https://www.sec.gov/edgar/browse/?CIK={CIK}&owner=exclude')
    RST()
    browser.find_element_by_xpath(r'/html/body/main/div[4]/div[2]/div[3]').click()
    
    time.sleep(3)
    browser.find_element_by_xpath(r'/html/body/main/div[4]/div[2]/div[3]/div/button[1]').click()
    
    time.sleep(3)
    how_much_pages = periods
    count_page = 0 
    for page in range(how_much_pages):
        count_page+=1

        browser.find_element_by_xpath(f'/html/body/main/div[5]/div/div[3]/div[3]/div[2]/table/tbody/tr[{count_page}]/td[2]/div/a[1]').click()
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(3)
        xml_report_urls.append(browser.current_url)
        time.sleep(3)
        browser.switch_to.window(browser.window_handles[0])

    return xml_report_urls

In [7]:
def recod_folder(tiker):

    basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports'
    full_path = basic_folder + '\\' + tiker
    try:

        os.mkdir(full_path)
    except:
        pass
    finally:
        pass
    return full_path

In [8]:
def find_wrong_value(current_url):
    find_wrog_things = current_url.split('/')
    clear_url = find_wrog_things[-2]
    if len(clear_url)>8:
        return current_url
    else:

        find_wrog_things.remove(find_wrog_things[-2])
        return '/'.join(find_wrog_things)


In [9]:
def convert_from_xmls(
    xml_report_urls, tiker
):  # pure urls ['https://www.sec.gov/Archives/edgar/data/0001013934/000101393421000008',
    recod_folder(tiker)
    reports_urls = []
    for urls in xml_report_urls:
        browser = selelenium(tiker)
        browser.implicitly_wait(3)
        
        
        bad_text = (urls[:77].replace(r'ix?doc=/', "") +
                    '/Financial_Report.xlsx')
        
        name_excel_url = find_wrong_value(bad_text)
        print(name_excel_url)   
        
        
        browser.get(name_excel_url)

       

    return reports_urls

In [10]:
def main_func(tiker):
    full_path = make_full_excel_path(tiker)
    try:
        books_list = [os.path.join(full_path, filename) for filename in os.listdir(full_path)]
        if len(books_list)==0:
            recod_folder(tiker) # функция для создания папки каталога excels
            cik_number = find_cik(tiker)
            found_all_reports = find_all_reports(cik_number)
            recod_folder(tiker)
            convert_from_xmls(found_all_reports,tiker)
    except:
        recod_folder(tiker) # функция для создания папки каталога excels
        cik_number = find_cik(tiker)
        found_all_reports = find_all_reports(cik_number)
        recod_folder(tiker)
        convert_from_xmls(found_all_reports,tiker)
#     recod_folder()
    
        books_list = [os.path.join(full_path, filename) for filename in os.listdir(full_path)]
    finally:
        return books_list

# Choose tiker
Just write Tiker, and the app will be returned excels list.

In [154]:
Tiker = 'OI'
main_func(Tiker)

['C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(1).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(10).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(11).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(12).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(13).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(14).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(15).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(2).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel

# Selection Reports

In [15]:
def check_threee_moth_ended(data_frame):
    
    count_ended = {}
    check_columns = []
    have_found =0 
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if re.search('Months Ended',i):
            count_ended[indexx] = i
            have_found +=1
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if have_found>0:
            data_frame.set_index
        
            try:
                
                new_columns = data_frame.columns[indexx] + ' ' +  second_row
 
            except:
                new_columns = data_frame.columns[indexx]

            check_columns.append(new_columns)

        else:
            
            check_columns.append(i)
            
    data_frame.columns  = check_columns
    if have_found >0:
        data_frame = data_frame.iloc[1:]

    else:
        pass

    
    return data_frame#.iloc[:,1:]

In [155]:
%%time

excl_add = 'Condensed'
exclude_report_list_before_join = ['Net Income per Share',
                       'Summary of Significant Accounting Policies','Common Stock'] 
exclude_report_list = '|'.join(exclude_report_list_before_join)
excludeded_keys = []

data_dict = defaultdict(list)   
REALY_NAMES =  {}
files = main_func(Tiker)
for period in files:
    
    wb = openpyxl.load_workbook(period)
    sheetnames_list = wb.sheetnames

    for sheetname in sheetnames_list:
        sheet = wb[sheetname]

        if len(str(sheet['b4'].value))<9 and len(str(sheet['b3'].value))<9:

            header = sheet['A1'].value
        
 

            if re.search(exclude_report_list,header):
                excludeded_keys.append(header)
                pass
            else :
                if re.search(excl_add,header):
                    
                    spisok = header.split(' ')
                    spisok.remove(excl_add)
                # (spisok.index(keysss))
                    header=' '.join(spisok)
                else:
                    pass

                REALY_NAMES[header] = sheetname
                a = check_threee_moth_ended(pd.read_excel(period,sheet_name=sheetname))
                
                data_dict.setdefault(header, []).append(a)

# print('OUT' ,len(excludeded_reports))
# for b,i in enumerate(excludeded_reports):
#     print(b+1,i)
# print('______________')

Wall time: 51.9 s


# Look at all selected Reports

In [156]:
print('OUTPUT',len(data_dict),'|||', 'INPUT',len(wb.sheetnames))   
for b,i in enumerate(REALY_NAMES.keys()):
    print(b+1,i)

OUTPUT 152 ||| INPUT 62
1 Document and Entity Information
2 CONDENSED CONSOLIDATED RESULTS OF OPERATIONS - USD ($) shares in Thousands, $ in Millions
3 CONDENSED CONSOLIDATED COMPREHENSIVE INCOME (LOSS) - USD ($) $ in Millions
4 CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions
5 CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical) - USD ($) $ in Millions
6 CONDENSED CONSOLIDATED CASH FLOWS - USD ($) $ in Millions
7 Pension Benefit Plans (Tables)
8 Segment Information (Details) - segment
9 Segment Information - Reportable Segments (Details) - USD ($) $ in Millions
10 Segment Information - Total Assets (Details) - USD ($) $ in Millions
11 Revenue (Details) - USD ($) $ in Millions
12 Credit Losses (Details) - USD ($) $ in Millions
13 Inventories (Details) - USD ($) $ in Millions
14 Derivative Instruments - Derivatives and Hedges (Details) - Cash Flow Hedges - USD ($) $ in Millions
15 Derivative Instruments - Balance Sheet Classification (Details) € in Millions, $ in Millions

# Concat Tables

In [162]:
# Just to see Reports where have 10 or more
[f'{frame_name} {len(data_dict[frame_name])}' for frame_name in REALY_NAMES.keys() if len(data_dict[frame_name])>10]

['CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions 12',
 'CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical) - USD ($) $ in Millions 12',
 'CONDENSED CONSOLIDATED CASH FLOWS - USD ($) $ in Millions 12',
 'Pension Benefit Plans (Tables) 11',
 'Segment Information - Total Assets (Details) - USD ($) $ in Millions 12',
 'Revenue (Details) - USD ($) $ in Millions 15',
 'Inventories (Details) - USD ($) $ in Millions 16',
 'Derivative Instruments - Balance Sheet Classification (Details) € in Millions, $ in Millions 14',
 'Derivative Instruments - Effects of Derivative Instruments (Details) - USD ($) $ in Millions 12',
 'Debt (Details) $ / shares in Units, € in Millions, $ in Millions 16',
 'Contingencies - Asbestos (Details) 16',
 'Accumulated Other Comprehensive Loss (Details) - USD ($) $ in Millions 11',
 'Earnings Per Share (Details) - USD ($) $ / shares in Units, $ in Millions 16',
 'Supplemental Cash Flow Information (Details) - USD ($) $ in Millions 16',
 'Segment Inform

In [171]:
# finder = Report name
finder = "CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions"
print(f'App have found {len(data_dict[finder])} documents from\n{REALY_NAMES[finder]}')

App have found 12 documents from
CONDENSED CONSOLIDATED BALANCE 


# Prepare_Data

In [163]:
from collections import Counter
def convert_column_index_to_unique(frame):
    war_dict = {}
    
    mylist = frame[frame.columns[0]].to_list()


    [war_dict.update({k:0}) for k,v in Counter(mylist).items()]

    pure_list = []
    for name in mylist:
#         name = re.sub('\d','',name)
        if name not in war_dict.keys():
            
            pure_list.append(name)
        else:
            
            war_dict[name] = war_dict[name]+1
            
            new_name = f'{name}{str(war_dict[name])}'
            pure_list.append(new_name.replace('  ',' '))
    return pure_list

In [164]:
def convert_to_finance_period_x(period_input):
    try:
        perods_dict = {'Nov':'Q_1 - 12','Oct':'Q_1 - 11','Sep':'Q_1 - 10','Aug':'Q_2 - 9','Jul':'Q_2 - 8','Jun':'Q_2 - 7','May':'Q_3 - 6','Apr':'Q_3 - 5','Mar':'Q_3 - 4','Feb':'Q_4 - 3','Dec':'Q_4 - 2','Jan':'Q_4 - 1'}
    
#         'Apr', 'Aug', 
        year = re.findall(r'\d\d\d\d',period_input)[0]
        quartal = perods_dict[re.findall(r'\D\D\D',period_input)[0]]
        
    
        return str(year + ' / ' + quartal)
    except:
        return period_input

In [165]:
def exceptions_rows_delete(frame):
    exceptions_rows = [
    'Common Stock$'
    ]
    list_for_delete = []
    for index,row in enumerate(frame.iloc[:,0].to_list()):
    
        for exception_r in exceptions_rows:
    
            if re.match(exception_r,row):
                print(index)
                list_for_delete.append(index)
            
        
        
    return list_for_delete

In [166]:
def cocncat_mean_rows(frame):
#     new_f = frame.fillna(0)
    
    for row in frame.iloc[:,0]:
        if int(re.search('\d',row)[0])>1:
            pass
        
#             print('here')
#     if sum([pd.Series(frame[col]) for col in new_f.columns[1:]])
    
    
    return frame

In [167]:
def pop_unneed_numbers(frame):
    
   
    withoud_number_list = [re.sub('\d','',row) for row in frame.iloc[:,0]]
    return withoud_number_list

In [168]:
def scrub_one_period(frame):
    frame.iloc[:,0] = pop_unneed_numbers(frame)
    frame.iloc[:,0] = convert_column_index_to_unique(frame) # assigned unque Value for 0 Column
    frame.columns = list(map(lambda date: convert_to_finance_period_x(date),frame.columns)) # convert for normal periods
    
#     w = exceptions_rows_delete(frame)
    return frame

In [169]:
def clear_set(ar):
    ar[ar.columns[0]] = ar[ar.columns[0]].apply(lambda x: x[0:75])
    
    return ar

In [170]:
def clear_frame(frame):
    if len(str(frame.columns[0]))<4 :
        print('here')   
        frame.index=frame.iloc[:,0]
        frame.index.name = 'Category'
        frame.drop(frame.columns[[0]],axis='columns',inplace=True)
    else:
        pass
    return frame
    

In [147]:
def concat_tables(key_name):
    frame_size = len(data_dict[key_name])-1
    structure = list(map(lambda dirty_drame:scrub_one_period(dirty_drame),data_dict[key_name]))
    def recurs(x):
       
        if x ==0:
            col_name = structure[x].columns[0]
            return (structure[x]).merge((structure[x+1]),on=key_name,how='outer',suffixes=('', '_y'))
        
            
        else:
            col_name = structure[x].columns[0]
            
            return recurs(x-1).merge((structure[x-1]),on=key_name,how='outer',suffixes=('', '_y'))
        
    frame = recurs(frame_size)
    
    try:
        
        if frame_size==1:
            return (structure[0])
        else:
            frame = recurs(frame_size)
            frame.drop(frame.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        
            return frame
    except Exception as ex:
        return ex
    


# GO

In [172]:
conc = concat_tables(finder)
new = conc[conc.columns[1:].sort_values(ascending=True)]
new.insert(0,conc.columns[0],conc[conc.columns[0]])

In [175]:
new

,CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions,2017 / Q_1 - 10,2017 / Q_2 - 7,2017 / Q_3 - 4,2017 / Q_4 - 2,2018 / Q_1 - 10,2018 / Q_2 - 7,2018 / Q_3 - 4,2018 / Q_4 - 2,2019 / Q_1 - 10,2019 / Q_2 - 7,2019 / Q_3 - 4,2019 / Q_4 - 2,2020 / Q_1 - 10,2020 / Q_2 - 7,2020 / Q_3 - 4,2020 / Q_4 - 2,2021 / Q_2 - 7,2021 / Q_3 - 4
0,Current assets:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cash and cash equivalents1,339.0,335.0,312.0,492.0,440.0,365.0,418.0,512.0,273.0,371.0,326.0,551.0,606.0,1067.0,891.0,563.0,531.0,742.0
2,"Trade receivables, net of allowance of $ million, $ million, and $ million at June , , December , and June , 1",NaN,956.0,NaN,NaN,NaN,1026.0,NaN,NaN,NaN,1275.0,NaN,NaN,NaN,696.0,NaN,623.0,855.0,NaN
3,Inventories1,1046.0,1049.0,1051.0,1036.0,973.0,985.0,1065.0,1018.0,1015.0,1015.0,1038.0,1045.0,782.0,1004.0,1047.0,841.0,796.0,827.0
4,Prepaid expenses and other current assets1,254.0,214.0,212.0,229.0,259.0,248.0,240.0,278.0,278.0,282.0,276.0,271.0,272.0,245.0,257.0,270.0,217.0,203.0
5,Total current assets1,2667.0,2554.0,2419.0,2420.0,2664.0,2624.0,2768.0,2357.0,2534.0,2943.0,2579.0,2488.0,2384.0,3012.0,2962.0,2297.0,2399.0,2486.0
6,"Property, plant and equipment, net1",3036.0,2996.0,2926.0,3131.0,3044.0,3025.0,3190.0,3085.0,3104.0,3216.0,3074.0,3273.0,2675.0,2976.0,2987.0,2907.0,2842.0,2791.0
7,Goodwill1,2621.0,2588.0,2524.0,2590.0,2549.0,2533.0,2649.0,2513.0,1893.0,2527.0,2507.0,1934.0,1847.0,1806.0,1773.0,1951.0,1932.0,1880.0
8,"Intangibles, net1",473.0,486.0,485.0,439.0,422.0,419.0,452.0,400.0,404.0,422.0,394.0,371.0,311.0,314.0,312.0,325.0,309.0,310.0
9,Other assets1,1202.0,1156.0,1105.0,1176.0,1261.0,1217.0,1222.0,1344.0,1613.0,1615.0,1598.0,1544.0,1407.0,1471.0,1470.0,1402.0,1392.0,1358.0
